In [19]:
import pandas as pd
import subprocess; FOLDER_PATH = subprocess.Popen(['git', 'rev-parse', '--show-toplevel'], stdout=subprocess.PIPE).communicate()[0].rstrip().decode('utf-8')
import sys, os; sys.path.append(os.path.dirname(f'{FOLDER_PATH}/utils')); sys.path.append(os.path.dirname(f'{FOLDER_PATH}/scripts'))

from utils.base_helpers import calculate_wmape

In [20]:
import warnings; warnings.filterwarnings("ignore")

In [21]:
df = pd.read_parquet("/home/mert/Desktop/thesis/data/processed/outlier_removed.parquet")

In [22]:
df = df[["forecast_dt", "rt_plant_id", "production"]]
df

,forecast_dt,rt_plant_id,production
0,2019-01-24 03:00:00,672,49.0
1,2019-01-24 04:00:00,672,59.0
2,2019-01-24 05:00:00,672,32.0
3,2019-01-24 06:00:00,672,34.0
4,2019-01-24 07:00:00,672,44.0
...,...,...,...
2481313,2022-01-27 19:00:00,2374,20.0
2481314,2022-01-27 20:00:00,2374,18.5
2481315,2022-01-27 21:00:00,2374,18.1
2481316,2022-01-27 22:00:00,2374,13.7


In [23]:
df.head()

,forecast_dt,rt_plant_id,production
0,2019-01-24 03:00:00,672,49.0
1,2019-01-24 04:00:00,672,59.0
2,2019-01-24 05:00:00,672,32.0
3,2019-01-24 06:00:00,672,34.0
4,2019-01-24 07:00:00,672,44.0


In [27]:
df["1h"] = df.groupby("rt_plant_id").production.shift(1)
df = df[df.forecast_dt.dt.year == 2021]

In [6]:
df["24h"] = df.groupby("rt_plant_id").production.shift(24)
df["48h"] = df.groupby("rt_plant_id").production.shift(48)
df = df[df.forecast_dt.dt.year == 2021]

In [28]:
df

,forecast_dt,rt_plant_id,production,1h
16989,2021-01-01 00:00:00,672,84.50,86.3
16990,2021-01-01 01:00:00,672,35.40,84.5
16991,2021-01-01 02:00:00,672,29.30,35.4
16992,2021-01-01 03:00:00,672,38.30,29.3
16993,2021-01-01 04:00:00,672,37.30,38.3
...,...,...,...,...
2480665,2021-12-31 19:00:00,2374,0.50,2.0
2480666,2021-12-31 20:00:00,2374,2.60,0.5
2480667,2021-12-31 21:00:00,2374,5.60,2.6
2480668,2021-12-31 22:00:00,2374,6.20,5.6


In [30]:
from sklearn.preprocessing import MinMaxScaler as scaler_
scalers = {}
lower_bound = 1e-8

for i, plant in enumerate(df.rt_plant_id.unique()):
    scalers[plant] = scaler_()
    # cols = ["production", "24h", "48h"]
    cols = ["production", "1h"]
    df.loc[df["rt_plant_id"] == plant, cols] = scalers[plant].fit_transform(df.loc[df["rt_plant_id"] == plant, cols]).clip(min=lower_bound, max=1-lower_bound)

In [ ]:
df_24 = df[["forecast_dt", "rt_plant_id", "production", "24h"]].rename(
    columns={"production": "actuals", "24h": "predictions"})
df_48 = df[["forecast_dt", "rt_plant_id", "production", "48h"]].rename(
    columns={"production": "actuals", "48h": "predictions"})

In [31]:
df_1 = df[["forecast_dt", "rt_plant_id", "production", "1h"]].rename(
    columns={"production": "actuals", "1h": "predictions"})

In [32]:
df_1.to_csv("/home/mert/Desktop/thesis/data/out/persistence1_output.csv", index=False)

In [12]:
df_24 = df[["forecast_dt", "rt_plant_id", "production", "24h"]].rename(
    columns={"production": "actuals", "24h": "predictions"})
df_48 = df[["forecast_dt", "rt_plant_id", "production", "48h"]].rename(
    columns={"production": "actuals", "48h": "predictions"})

In [16]:
df_24.to_csv("/home/mert/Desktop/thesis/data/out/persistence24_output.csv", index=False)

In [17]:
df_48.to_csv("/home/mert/Desktop/thesis/data/out/persistence48_output.csv", index=False)

In [18]:
df_24

,forecast_dt,rt_plant_id,actuals,predictions
16989,2021-01-01 00:00:00,672,0.965714,0.950743
16990,2021-01-01 01:00:00,672,0.404571,0.990857
16991,2021-01-01 02:00:00,672,0.334857,0.963429
16992,2021-01-01 03:00:00,672,0.437714,0.985143
16993,2021-01-01 04:00:00,672,0.426286,0.993143
...,...,...,...,...
2480665,2021-12-31 19:00:00,2374,0.025000,0.295000
2480666,2021-12-31 20:00:00,2374,0.130000,0.240000
2480667,2021-12-31 21:00:00,2374,0.280000,0.315000
2480668,2021-12-31 22:00:00,2374,0.310000,0.170000
